# Install additional Python libraries

In [1]:
!pip install quantities

# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


# Load Agent model

In [6]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [7]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [8]:
AGENTS = {
    "human_agent": [{"connections": {"air_storage": [1], 'water_storage': [1, 2], "nutrient_storage": [1], "power_storage": [1], "food_storage": [1]}, "amount": 2}],
    "cabbage": [{"connections": {"air_storage": [1], 'water_storage': [1, 2], "nutrient_storage": [1], "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], 'water_storage': [1, 2], "nutrient_storage": [1], "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}],
}

STORAGES = {
    "air_storage": [{"id": 1, "atmo_h2o":100, "atmo_o2": 100, "atmo_co2": 100, "atmo_h2o": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 100}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100, "sold_wast": 0}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000, "heat_cal": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 200}]
}

game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(AGENTS, STORAGES)
game_runner_manager.new_game(user, game_runner_init_params)

# Run the simiulation

In [9]:
num_steps = 10
for i in range(1, num_steps):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)

self.deprive[h2o_potb]: 1.9583333333333333
{'agents': [{'agent_type': 'solar_pv_array',
             'attributes': {'age': 0.3333333333333333},
             'id': 'GeneralAgent_dce4e718-38d0-4928-a389-61e9599a30ee'},
            {'agent_type': 'solar_pv_array',
             'attributes': {'age': 0.3333333333333333},
             'id': 'GeneralAgent_d845f4b3-56c1-4d05-9c3d-01d9473ffd15'},
            {'agent_type': 'solar_pv_array',
             'attributes': {'age': 0.3333333333333333},
             'id': 'GeneralAgent_72c5670c-b0b1-47c3-b33b-447a92cf1e9a'},
            {'agent_type': 'cabbage',
             'attributes': {'age': 0.3333333333333333},
             'id': 'GeneralAgent_cdc05672-288d-4224-94f9-9c8b8742250b'},
            {'agent_type': 'lettuce',
             'attributes': {'age': 0.3333333333333333},
             'id': 'GeneralAgent_321d5290-0717-4c1e-9440-9604ec10816e'},
            {'agent_type': 'solar_pv_array',
             'attributes': {'age': 0.3333333333333333},


In [10]:
game_runner_manager.save_all(allow_repeat_save=False)